In [1]:
import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset,random_split
from torchvision import datasets, transforms

import pandas as pd
import numpy as np

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

/home/viki/miniconda3/envs/WaterLevelPrediction/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


'cpu'

In [3]:
data = pd.read_csv("csv/dummyData-1.csv",header = None)
x=data.iloc[1:,1:5].astype('float32')
y=data.iloc[1:,-1].astype('float32')
# x.iloc[:, 0] = x.iloc[:, 0] / 23
x.iloc[:, 2] = x.iloc[:, 2] / 100
x.iloc[:, 3] = x.iloc[:, 3] / 100
y = y/max(y.values)

In [29]:
x

,1,2,3,4
1,1.0,1.0,0.94,0.11
2,0.0,1.0,0.54,0.70
3,0.0,1.0,0.36,0.06
4,1.0,1.0,0.66,0.74
5,0.0,1.0,0.49,0.97
...,...,...,...,...
99996,1.0,1.0,0.93,0.59
99997,1.0,1.0,0.13,0.69
99998,1.0,1.0,0.10,0.69
99999,0.0,1.0,0.64,0.02


In [7]:
class WaterLevelDataset(Dataset):
    def __init__(self, x, y):
        x= torch.tensor(x,dtype=torch.float32)
        y = torch.tensor(y,dtype=torch.float32)
        self.X = x
        self.Y = y

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.Y[idx]]

In [8]:
dataset = WaterLevelDataset(x.values,y.values)

In [9]:
len(dataset.X)

100000

In [10]:
dataset.X

tensor([[1.0000, 1.0000, 0.9400, 0.1100],
        [0.0000, 1.0000, 0.5400, 0.7000],
        [0.0000, 1.0000, 0.3600, 0.0600],
        ...,
        [1.0000, 1.0000, 0.1000, 0.6900],
        [0.0000, 1.0000, 0.6400, 0.0200],
        [0.0000, 0.0000, 0.4400, 0.5200]])

In [11]:
dataset.Y

tensor([1.0000, 0.2000, 0.8000,  ..., 0.0000, 0.8000, 0.2000])

In [12]:
train, test = random_split(dataset, [int(len(dataset.X)*0.8),int(len(dataset.X)*0.2)])

In [13]:
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

## Model 1
- Dataset used here is a dummy data with if top level more then 70 then predition should be 0

In [12]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer = nn.Linear(4, 10)
        self.layer2 = nn.Linear(10,1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = torch.relu(self.layer(X))
        X=self.layer2(X)
        X = self.activation(X)
        return X

# Instantiate the model
model = SimpleNN().to(device)



In [13]:
loss_fn = nn.MSELoss().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001)

In [14]:
for epoch in range(100):  # Number of epochs
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs.squeeze(), targets.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/100], Loss: {loss.item()*100:0f}")



Epoch [1/100], Loss: 12.345994
Epoch [2/100], Loss: 7.209603
Epoch [3/100], Loss: 9.753313


KeyboardInterrupt: 

In [35]:
model.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            test_inputs = test_inputs.to(device)
            outputs = model(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx].to(device))
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {int(outputs.item()*5:.2f}, Actual: {targets[idx].item()*5:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")


Predicted: 1, Actual: 3.00
Predicted: 4, Actual: 4.00
Predicted: 0, Actual: 0.00
Predicted: 4, Actual: 5.00
Predicted: 0, Actual: 0.00
Predicted: 1, Actual: 0.00
Predicted: 0, Actual: 0.00
Predicted: 4, Actual: 4.00
Predicted: 4, Actual: 4.00
Predicted: 1, Actual: 3.00
Predicted: 0, Actual: 0.00
Predicted: 0, Actual: 0.00
Predicted: 1, Actual: 1.00
Predicted: 1, Actual: 0.00
Predicted: 2, Actual: 4.00
Predicted: 1, Actual: 0.00
Average loss on the testing dataset: 2.5973
Predicted: 4, Actual: 5.00
Predicted: 0, Actual: 0.00
Predicted: 3, Actual: 4.00
Predicted: 2, Actual: 3.00
Predicted: 4, Actual: 5.00
Predicted: 0, Actual: 0.00
Predicted: 0, Actual: 2.00
Predicted: 4, Actual: 5.00
Predicted: 2, Actual: 2.00
Predicted: 0, Actual: 0.00
Predicted: 4, Actual: 4.00
Predicted: 1, Actual: 0.00
Predicted: 3, Actual: 3.00
Predicted: 4, Actual: 5.00
Predicted: 3, Actual: 3.00
Predicted: 0, Actual: 0.00
Average loss on the testing dataset: 2.0290
Predicted: 4, Actual: 4.00
Predicted: 1, Actual:

In [ ]:
torch.save(model, "model/model1Data-1-10000-10.h5")

## Model 2
- Dataset used here is a dummy data with if top level more then 70 then predition should be 0

In [5]:
class Model2NN(nn.Module):
    def __init__(self):
        super(Model2NN, self).__init__()
        self.layer = nn.Linear(4, 10)
        self.layer2 = nn.Linear(10, 10)
        self.layer3 = nn.Linear(10,1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = torch.relu(self.layer(X))
        X=torch.relu(self.layer2(X))
        X=self.layer3(X)
        X = self.activation(X)
        return X

# Instantiate the model
model1 = Model2NN().to(device)



In [16]:
loss_fn = nn.MSELoss().to(device)

optimizer = optim.SGD(model1.parameters(), lr=0.001)

In [31]:
for epoch in range(250):  # Number of epochs
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model1(inputs)
        loss = loss_fn(outputs.squeeze(), targets.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/100], Loss: {loss.item()*100:0f}")



Epoch [1/100], Loss: 2.411921
Epoch [2/100], Loss: 2.664525
Epoch [3/100], Loss: 5.762581
Epoch [4/100], Loss: 2.816133
Epoch [5/100], Loss: 9.949568
Epoch [6/100], Loss: 3.162954
Epoch [7/100], Loss: 2.222726
Epoch [8/100], Loss: 2.018509
Epoch [9/100], Loss: 3.216168
Epoch [10/100], Loss: 2.628627
Epoch [11/100], Loss: 2.961287
Epoch [12/100], Loss: 4.807064
Epoch [13/100], Loss: 9.774303
Epoch [14/100], Loss: 3.810337
Epoch [15/100], Loss: 3.237595
Epoch [16/100], Loss: 2.339025
Epoch [17/100], Loss: 2.576985
Epoch [18/100], Loss: 4.306972
Epoch [19/100], Loss: 3.729443
Epoch [20/100], Loss: 5.399982
Epoch [21/100], Loss: 6.419381
Epoch [22/100], Loss: 5.614897
Epoch [23/100], Loss: 3.081389
Epoch [24/100], Loss: 2.653601
Epoch [25/100], Loss: 4.093307
Epoch [26/100], Loss: 3.105984
Epoch [27/100], Loss: 2.962005
Epoch [28/100], Loss: 4.660344
Epoch [29/100], Loss: 4.463029
Epoch [30/100], Loss: 5.596363
Epoch [31/100], Loss: 3.812166
Epoch [32/100], Loss: 1.928401
Epoch [33/100], L

In [32]:
model1.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            test_inputs = test_inputs.to(device)
            outputs = model1(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            # loss = loss_fn(torch.teint(outputs.squeeze()*5)), torch.tensor(int(targets[idx].to(device)*5)))
            loss = int(outputs.item()*5) == int(targets[idx].item()*5)

            # Accumulate the lossmodel1.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            test_inputs = test_inputs.to(device)
            outputs = model1(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx].to(device))
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {int(outputs.item()*5)}, Actual: {targets[idx].item()*5:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")
            total_loss += not loss
            num_samples += 1
            
            # Print the predictions and the actual targets
            # print(f"Predicted: {int(outputs.item()*5)}, Actual: {targets[idx].item()*5:.2f}")
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")

Predicted: 1, Actual: 3.00
Predicted: 4, Actual: 4.00
Predicted: 0, Actual: 0.00
Predicted: 4, Actual: 5.00
Predicted: 0, Actual: 0.00
Predicted: 1, Actual: 0.00
Predicted: 0, Actual: 0.00
Predicted: 4, Actual: 4.00
Predicted: 4, Actual: 4.00
Predicted: 1, Actual: 3.00
Predicted: 0, Actual: 0.00
Predicted: 0, Actual: 0.00
Predicted: 1, Actual: 1.00
Predicted: 1, Actual: 0.00
Predicted: 2, Actual: 4.00
Predicted: 1, Actual: 0.00
Average loss on the testing dataset: 43.7500
Predicted: 4, Actual: 5.00
Predicted: 0, Actual: 0.00
Predicted: 3, Actual: 4.00
Predicted: 2, Actual: 3.00
Predicted: 4, Actual: 5.00
Predicted: 0, Actual: 0.00
Predicted: 0, Actual: 2.00
Predicted: 4, Actual: 5.00
Predicted: 2, Actual: 2.00
Predicted: 0, Actual: 0.00
Predicted: 4, Actual: 4.00
Predicted: 1, Actual: 0.00
Predicted: 3, Actual: 3.00
Predicted: 4, Actual: 5.00
Predicted: 3, Actual: 3.00
Predicted: 0, Actual: 0.00
Average loss on the testing dataset: 46.8750
Predicted: 4, Actual: 4.00
Predicted: 1, Actua

In [37]:
model1.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            test_inputs = test_inputs.to(device)
            outputs = model1(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx].to(device))
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            # print(f"Predicted: {int(outputs.item()*5)}, Actual: {targets[idx].item()*5:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")

Average loss on the testing dataset: 2.5973
Average loss on the testing dataset: 2.0290
Average loss on the testing dataset: 2.7055
Average loss on the testing dataset: 2.3487
Average loss on the testing dataset: 2.1953
Average loss on the testing dataset: 1.9098
Average loss on the testing dataset: 1.8821
Average loss on the testing dataset: 1.9312
Average loss on the testing dataset: 1.8272
Average loss on the testing dataset: 1.9672
Average loss on the testing dataset: 2.2741
Average loss on the testing dataset: 2.4228
Average loss on the testing dataset: 2.4462
Average loss on the testing dataset: 2.4212
Average loss on the testing dataset: 2.3722
Average loss on the testing dataset: 2.3194
Average loss on the testing dataset: 2.3543
Average loss on the testing dataset: 2.2657
Average loss on the testing dataset: 2.2416
Average loss on the testing dataset: 2.1886
Average loss on the testing dataset: 2.1902
Average loss on the testing dataset: 2.2206
Average loss on the testing data

In [38]:
torch.save(model1, "model/model2Data-1-10000-10p-t.pt")

In [24]:
test1 = torch.tensor([0,0,90/100,20/100]).to(device)
print(test1)
outputs = load_model(test1)
print(int(outputs.item()*5))

tensor([0.0000, 0.0000, 0.9000, 0.2000], device='cuda:0')
3


## Testing Model

In [6]:
load_model = torch.load("model/model2Data-1-10000-10pt.pt",map_location=torch.device(device))

In [31]:
load_model.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            test_inputs = test_inputs.to(device)
            outputs = load_model(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            # loss = loss_fn(torch.teint(outputs.squeeze()*5)), torch.tensor(int(targets[idx].to(device)*5)))
            loss = int(outputs.item()*5) == int(targets[idx].item()*5)

            # Accumulate the loss
            total_loss += not loss
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {int(outputs.item()*5)}, Actual: {targets[idx].item()*5:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [7]:
# Save the state dictionary
torch.save(load_model.state_dict(), "model/model2Data-1-10000-10pt_state.pt")
